In [1]:
import shutil
from lxml import etree
from glob import glob
import os
import logging
import json
import pandas as pd
import re
from datetime import datetime

# For scraping
import requests
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup
from datetime import datetime

## Initial Conf Details

In [2]:
# Basic setup
now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M%S")

JRN_CODE = 'afar'
LICENCE_FOLDER = 'license_info'
TEST_FOLDER = 'test_input'
ERROR_FOLDER = 'logs'
logging.basicConfig(filename='logs/log_file_{}_{}.log'.format(JRN_CODE, dt_string), filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logging.info('running')


# Code for Scraping Atypon Site ####
## (Skip this section if you already have the CSV files) 

In [41]:
driver = webdriver.Chrome("/Users/kmcdouga/Dropbox (MIT)/Silverchair/Batch_2/venv/bin/chromedriver")

In [42]:
loi_url = 'https://www.mitpressjournals.org/loi/{}'.format(JRN_CODE)
driver.get(loi_url)
content = driver.page_source
soup = BeautifulSoup(content)


In [43]:
loi_content = soup.find_all('div', {'class':'js_issue'})
urls = []
for x in loi_content:
    link_tag =  x.find('a', recursive=False)['href']
    full_url = 'https://www.mitpressjournals.org{}'.format(link_tag)
    urls.append(full_url)

In [44]:
urls

['https://www.mitpressjournals.org/toc/daed/149/4',
 'https://www.mitpressjournals.org/toc/daed/149/3',
 'https://www.mitpressjournals.org/toc/daed/149/2',
 'https://www.mitpressjournals.org/toc/daed/149/1',
 'https://www.mitpressjournals.org/toc/daed/148/4',
 'https://www.mitpressjournals.org/toc/daed/148/3',
 'https://www.mitpressjournals.org/toc/daed/148/2',
 'https://www.mitpressjournals.org/toc/daed/148/1',
 'https://www.mitpressjournals.org/toc/daed/147/4',
 'https://www.mitpressjournals.org/toc/daed/147/3',
 'https://www.mitpressjournals.org/toc/daed/147/2',
 'https://www.mitpressjournals.org/toc/daed/147/1',
 'https://www.mitpressjournals.org/toc/daed/146/4',
 'https://www.mitpressjournals.org/toc/daed/146/3',
 'https://www.mitpressjournals.org/toc/daed/146/2',
 'https://www.mitpressjournals.org/toc/daed/146/1',
 'https://www.mitpressjournals.org/toc/daed/145/4',
 'https://www.mitpressjournals.org/toc/daed/145/3',
 'https://www.mitpressjournals.org/toc/daed/145/2',
 'https://ww

In [45]:
def scrape_toc():
    for url in urls:
        driver.get(url)
        content = driver.page_source
        soup = BeautifulSoup(content)
        doi_list, access_list = parse_html(soup, url)
    return doi_list, access_list

In [46]:
access_list = []
doi_list = []

In [47]:
master_list = []
def parse_html(soup, url):
    try:
        # We want to grab the tocContent element
        toc_content = soup.find("div", {"class": "tocContent"})
        article_entry = toc_content.find_all("table", {"class": "articleEntry"}, recursive=False)
        for article in article_entry:
            access_list.append(article.find("img", {"class": "accessIcon"}, recursive=True)['title'])
            doi_list.append(article.find("input", {"class": "tocToolCheckBox"}, recursive=True)['value'])
        return doi_list, access_list
    except NoneType as ne:
        print(ne)
        pass


In [48]:
doi_list, access_list = scrape_toc()
zipped = list(zip(doi_list, access_list))
df = pd.DataFrame(zipped,  columns=['DOI', 'ACCESS'])



In [49]:
driver.quit()

In [50]:
# Now that we've scraped all the file info
# We generate  a DF that holds all the access data and adds a new column for normalized article IDs

In [51]:
def normalize_doi(row):
    full_doi = row['DOI']
    normalized_doi = full_doi.split('/')[1].upper()
    return normalized_doi

df['NORMLIZED_DOI'] = df.apply(normalize_doi, axis=1)

# Save everything to a file, just in case
df.to_csv('{}/access_list_{}.csv'.format(LICENCE_FOLDER, JRN_CODE), index=None)


# Code for Checking and Updating the XML files ####

In [3]:
license_df = pd.read_csv('{}/access_list_{}.csv'.format(LICENCE_FOLDER, JRN_CODE))
print('loaded: {}'.format(license_df))

loaded:                        DOI     ACCESS NORMLIZED_DOI
0     10.1162/afar_a_00545  No Access  AFAR_A_00545
1     10.1162/afar_a_00546  No Access  AFAR_A_00546
2     10.1162/afar_a_00547  No Access  AFAR_A_00547
3     10.1162/afar_a_00548  No Access  AFAR_A_00548
4     10.1162/afar_a_00549  No Access  AFAR_A_00549
...                    ...        ...           ...
4863       10.2307/3334364  No Access       3334364
4864       10.2307/3334365  No Access       3334365
4865       10.2307/3334366  No Access       3334366
4866       10.2307/3334367  No Access       3334367
4867       10.2307/3334368  No Access       3334368

[4868 rows x 3 columns]


In [4]:
# Filter the license_df to only include OA articles
# Filter the license_df to only include Free article
# The gated articles don't require any changes.
oa_articles = license_df[license_df['ACCESS']=='Open Access']
free_articles = license_df[license_df['ACCESS']=='Free Access']


In [5]:
print(oa_articles)
print(free_articles)

Empty DataFrame
Columns: [DOI, ACCESS, NORMLIZED_DOI]
Index: []
                           DOI       ACCESS      NORMLIZED_DOI
7         10.1162/afar_a_00552  Free Access       AFAR_A_00552
17        10.1162/afar_a_00536  Free Access       AFAR_A_00536
28        10.1162/afar_a_00525  Free Access       AFAR_A_00525
37        10.1162/afar_a_00511  Free Access       AFAR_A_00511
49        10.1162/afar_a_00495  Free Access       AFAR_A_00495
55        10.1162/afar_a_00501  Free Access       AFAR_A_00501
63        10.1162/afar_a_00473  Free Access       AFAR_A_00473
64        10.1162/afar_a_00474  Free Access       AFAR_A_00474
67        10.1162/afar_a_00478  Free Access       AFAR_A_00478
89        10.1162/afar_a_00456  Free Access       AFAR_A_00456
90        10.1162/afar_a_00457  Free Access       AFAR_A_00457
105       10.1162/afar_a_00440  Free Access       AFAR_A_00440
111       10.1162/afar_a_00446  Free Access       AFAR_A_00446
120       10.1162/afar_x_00428  Free Access       AFAR

In [6]:
# In order for lxml to process the files we need to provide the below info
nsmap = {
'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
'xlink': 'http://www.w3.org/1999/xlink',
}
attr_qname = etree.QName("http://www.w3.org/2001/XMLSchema-instance", "schemaLocation")

parser = etree.XMLParser(recover=True)

In [7]:
# First we look for the issue_meta file and update the TOC as needed
# Next we look for OA and Free articles and add the license info as needed

def update_files():
    for root, directory, files in os.walk(TEST_FOLDER):
        for f in files:
            # If we have the issue_meta file we need to update it.
            # First we grab the path to the issue_meta file.
            # We then get df_filter, which holds all the DOIs in this issue
            # Then we get unique_headings, which holds the TOC headins if any exist. 
            # That info has been scraped from the website directly and lives in a CSV file
            # that we get from get_doi_list(). 
            # Once we have that we call convert_issue_xml(), which does the actual updating of the issue files
            if 'issue_meta' in f:
                path_to_issue_file = '{}/{}'.format(root, f)
#                 path_to_assets_folder = '{}/{}'.format(root, )
                issue_folder_name = root.split('/')[1]
                df_filter, unique_headings = get_doi_list('{}/{}/XML'.format(TEST_FOLDER, issue_folder_name))
                converted = convert_issue_xml(path_to_issue_file, df_filter, unique_headings, issue_folder_name)
#                 with open('{}'.format(path_to_issue_file), 'wb') as doc:
#                     doc.write(etree.tostring(converted, pretty_print = True))
            
            # If we have an article file we check to see if the license needs to added
            # Since the articles are named according to their DOI we can iterate over all the articles 
            # in each package and updated those in the filtered lists
            
            # This part is a bit inefficent and could do with some cleanup so that
            # we don't have two functions doing more or less the same thing. 
            # For now, however, it works. 
            if f.endswith('xml') and 'issue' not in f:
                article_path = '{}/{}'.format(root, f)
#                 add_review_title(article_path)
#                 add_jstor_link(article_path)
                article_name = f.split('.xml')[0].upper()
#                 if article_name.upper() in oa_articles['NORMLIZED_DOI'].values:
#                     update_license_oa(article_path)
#                     print(article_path)
#                 if article_name.upper() in free_articles['NORMLIZED_DOI'].values:
#                     update_license_free(article_path)

In [8]:
def update_license_oa(article_path):
    tree = etree.parse(article_path, parser=parser)
    article_root  = tree.getroot()
    # Look for an existing license element
    license_tag = tree.find('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}license')
    # If we find a license tag in the article we want to make sure it has the right license. 
    # If it hase a license element it SHOULD have the correct license, but we should check anyway
    # If it has the wrong license (anything other than 'open-access') just log it for review later
    if license_tag is not None:             
        if license_tag.get('license-type') != 'open-access':
            logging.info('File: {}. license found with incorrect license-type. Expectng \'open-access\' found \'{}\''.format(article_path, license_tag.get('license-type')))
        if license_tag.get('license-type') == 'open-access':
            logging.info('File: {}. license found with correct license-type.'.format(article_path))

    # We now have an OA ile without a license in the XML.
    # So that means we have to add it
    # We do that by finding the permission tag and appending a child
    else:
        permissions_tag = tree.find('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}permissions')
        new_license_tag = etree.SubElement(permissions_tag, 'license')
        new_license_tag.attrib['license-type'] = 'open-access'
        new_license_tag.attrib['{%s}href' % nsmap['xlink']] = 'https://creativecommons.org/licenses/by/4.0/'

        new_license_p = etree.SubElement(new_license_tag, 'license-p')
        new_license_p.text = 'This is an open-access article distributed under the terms of the Creative Commons Attribution 4.0 International License, which permits unrestricted use, distribution, and reproduction in any medium, provided the original work is properly cited. For a full description of the license, please visit <ext-link ext-link-type="uri" xlink:href="https://creativecommons.org/licenses/by/4.0/legalcode">https://creativecommons.org/licenses/by/4.0/legalcode</ext-link>.'

        logging.info('File: {}. Updating file with OA license.'.format(article_path))

        with open(article_path, 'wb') as doc:
            doc.write(etree.tostring(tree, pretty_print = True))
        with open(article_path, 'r+') as f:
            text = f.read()
            text = re.sub('&lt;ext-link ext-link-type="uri" xlink:href="https://creativecommons.org/licenses/by/4.0/legalcode"&gt;https://creativecommons.org/licenses/by/4.0/legalcode&lt;/ext-link&gt;', '<ext-link ext-link-type="uri" xlink:href="https://creativecommons.org/licenses/by/4.0/legalcode">https://creativecommons.org/licenses/by/4.0/legalcode</ext-link>', text)
            f.seek(0)
            f.write(text)
            f.truncate()

                   

In [9]:
def update_license_free(article_path):
    tree = etree.parse(article_path, parser=parser)
    article_root  = tree.getroot()

    license_tag = tree.find('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}license')
    if license_tag is not None:             
        logging.info('File: {}. No update to free article. license found with some license. Check before updatingcorrect license-type.'.format(article_path))
    else:
        permissions_tag = tree.find('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}permissions')
        new_license_tag = etree.SubElement(permissions_tag, 'license')
        new_license_tag.attrib['license-type'] = 'free'                        

        logging.info('File: {}. Updating file with free license.'.format(article_path))

        with open(article_path, 'wb') as doc:
            doc.write(etree.tostring(tree, pretty_print = True))

In [21]:
def convert_issue_xml(path_to_issue_file, df_filter, unique_headings, issue_folder_name):

    folder = os.path.dirname(os.path.dirname(path_to_issue_file)).split('/')[1]
    assets_folder = '{}/{}'.format(os.path.dirname(os.path.dirname(path_to_issue_file)), 'assets')
    issue_num = folder.split('-')[1]
    
    tree = etree.parse(path_to_issue_file, parser=parser)
    root = tree.getroot()

    # Some issues are missing their covers and we need to download them from the site
    # This checks if a cover exists and if it doesn't it grabs it
    def get_cover():
        if glob('{}/*cover.*'.format(assets_folder)):
            print('cover found in {}'.format(path_to_issue_file))
            pass
        else:
            print('cover missing')
            # Check if webdriver running
            try:
                driver.title
                print(True)
            except NameError as e:
                print(False)
                driver = webdriver.Chrome("/Users/kmcdouga/Dropbox (MIT)/Silverchair/Batch_2/venv/bin/chromedriver")

            # The URL to the issue homepage follows a pattern that we can stitch together using the volume and issue
            volume = root.find('.//{http://specifications.silverchair.com/xsd/1/21/SCJATS-journalissue.xsd}volume').text
            issue = root.find('.//{http://specifications.silverchair.com/xsd/1/21/SCJATS-journalissue.xsd}issue').text
            toc_path = 'https://www.mitpressjournals.org/toc/{}/{}/{}'.format(JRN_CODE, volume, issue)
            # toc_path = 'https://www.mitpressjournals.org/toc/{}/{}'.format(JRN_CODE, issue)
#             toc_path = 'https://www.mitpressjournals.org/toc/{}/{}'.format(JRN_CODE, volume)

            try:
                driver.get(toc_path)
                content = driver.page_source
                soup = BeautifulSoup(content, features='lxml')
            
                cover_tag = soup.find_all('img', {'alt':'Publication Cover'})
                cover_path = 'https://www.mitpressjournals.org/{}'.format(cover_tag[0]['src'])
                cover_name = cover_path.split('/')[-1]
            
                supplementary_material = etree.SubElement(root, 'supplementary-material')
                supplementary_material.attrib['{%s}href' % nsmap['xlink']] = cover_name
                supplementary_material.attrib['content-type'] = 'cover'
            
                response = requests.get(cover_path)
                save_path = '{}/{}'.format(assets_folder, cover_name)
                file = open(save_path, "wb")
                file.write(response.content)
                file.close()
                driver.quit()
            except Exception as e:
                print('error getting cover: {}'.format(e))
                try:
                    driver.quit()
                except Exception as e:
                    print(e)

    get_cover()
    # issue_identifier = root.find('.//{http://specifications.silverchair.com/xsd/1/21/SCJATS-journalissue.xsd}issue-identifier')
    # vol = etree.SubElement(issue_identifier, 'volume')
    # vol.attrib['content-type'] = 'empty'

    # issue = etree.SubElement(issue_identifier, 'issue')
    # issue.text = issue_num

    
    
    # Now we update the issue_meta file adding the <toc> block
    toc_block = root.find('.//{http://specifications.silverchair.com/xsd/1/21/SCJATS-journalissue.xsd}table-of-contents')
    toc_block.getparent().remove(toc_block)


    table_of_contents = etree.SubElement(root, 'table-of-contents')
    
    # Look for articles without a heading in the TOC
    no_heading = df_filter[pd.isna(df_filter['article_title'])]

    
#     toc_sec = etree.SubElement(table_of_contents, 'toc-sec')
#     toc_title = etree.SubElement(toc_sec, 'title')

    if not no_heading.empty:
        toc_sec = etree.SubElement(table_of_contents, 'toc-sec')
        toc_title = etree.SubElement(toc_sec, 'title')
        for index, row in no_heading.iterrows():
            toc_content = etree.SubElement(toc_sec, 'toc-content')
            toc_content.attrib['pub-id-type'] = 'doi'
            toc_content.attrib['{%s}href' % nsmap['xlink']] = row['article_doi']

    for heading in unique_headings:
        toc_sec = etree.SubElement(table_of_contents, 'toc-sec')
        toc_title = etree.SubElement(toc_sec, 'title')
        toc_title.text = heading
        df_sec = df_filter[df_filter.article_title == heading]
        for index, row in df_sec.iterrows():
            toc_content = etree.SubElement(toc_sec, 'toc-content')
            toc_content.attrib['pub-id-type'] = 'doi'
            toc_content.attrib['{%s}href' % nsmap['xlink']] = row['article_doi']
    return root

def get_doi_list(path):
    delete_command = 'find \\{} -name \'*.DS_Store*\' -delete'.format(path)
    os.system(delete_command)

    doi_list = ['10.1162/{}'.format(os.path.splitext(item)[0].upper()) for item in os.listdir(path)]
#     doi_list = ['10.2307/{}'.format(os.path.splitext(item)[0].upper()) for item in os.listdir(path)]

    
    df = pd.read_csv('{}.csv'.format(JRN_CODE))

    def popTime(row):
        x = row['article_doi']
        return(x.upper())

    df['article_doi'] = df.apply(popTime, axis=1)
    df_filter = df[df.article_doi.isin(doi_list)]

    unique_headings = pd.unique(df_filter['article_title'])
    unique_headings = [value for value in unique_headings if pd.notna(value)]

    return df_filter, unique_headings



In [11]:
def add_jstor_link(article_path):
    print(article_path)
    tree = etree.parse(article_path)
    article_root  = tree.getroot()

    article_id_tags = tree.findall('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}article-id')

    for tag in article_id_tags:
        if tag.get("pub-id-type") == 'doi':
            permissions_tag = tree.find('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}permissions')

            abs_tag = etree.Element("abstract")
            permissions_tag.addnext(abs_tag)

            p_tag = etree.SubElement(abs_tag, 'p')
            p_tag.text = 'Available on JSTOR at: doi.org/{}'.format(tag.text)

        if tag.get("pub-id-type") == 'jstor':
            comment = etree.Comment(' === <article-id pub-id-type="jstor">{}</article-id> === '.format(tag.text))
            article_id_tags[0].addnext(comment)

            tag.getparent().remove(tag)
            
    with open(article_path, 'wb') as doc:
        doc.write(etree.tostring(tree, pretty_print = True))


In [13]:
def add_review_title(article_path):
    tree = etree.parse(article_path, parser=parser)
    article_root  = tree.getroot()

    if article_root.get('article-type') == 'other' or article_root.get('article-type') == 'book-review' or article_root.get('article-type') == 'review-article':

        try:
            # See if there is an article title
            article_title_tag = tree.xpath('///xsi:article/xsi:front/xsi:article-meta/xsi:title-group/xsi:article-title', namespaces={'xsi':'http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd'})
            # Check if there is a product > source tag
            product_source_tags = tree.findall('.//{http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd}source')
            source_tags_text = [ tag.text for tag in product_source_tags ]
            # Check if there is a product > article-title tag
            review_title_tags = tree.xpath('///xsi:article/xsi:front/xsi:article-meta/xsi:product/xsi:article-title', namespaces={'xsi':'http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd'})
            review_title_tags_text = [ tag.text for tag in review_title_tags ]
            # Check if there is a product > article-title >italic tag
            review_title_italic_tags = tree.xpath('///xsi:article/xsi:front/xsi:article-meta/xsi:product/xsi:article-title/xsi:italic', namespaces={'xsi':'http://specifications.silverchair.com/xsd/1/22/SCJATS-journalpublishing.xsd'})
            review_title_italic_tags_text = [ tag.text for tag in review_title_italic_tags ]

            # If the article-title tag is empty
            if article_title_tag[0].text == None:
                if len(review_title_italic_tags) != 0:
                    title_text = "; ".join(review_title_italic_tags)
                    article_title_tag[0].text = title_text
                    print(article_path)
                    print(title_text)
                elif len(review_title_tags_text) != 0:
                    title_text = "; ".join(review_title_tags_text)
                    article_title_tag[0].text = title_text
                    print(article_path)
                    print(title_text)
                elif len(source_tags_text) != 0:
                    title_text = "; ".join(source_tags_text)
                    article_title_tag[0].text = title_text
                    print(article_path)
                    print(title_text)
                else:
                    print('no product element in: {}'.format(article_path))
        except Exception as e:
            print('{} in {}'.format(e, article_path))

#     with open(article_path, 'wb') as doc:
#         doc.write(etree.tostring(tree, pretty_print = True))


In [23]:
update_files()

cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover found in test_input/afar-40-1/xml/issue_meta.xml
cover found in test_input/afar-40-2/xml/issue_meta.xml
cover found in test_input/afar-40-3/xml/issue_meta.xml
cover found in test_input/afar-40-4/xml/issue_meta.xml
cover found in test_input/afar-41-1/xml/issue_meta.xml
cover found in test_input/afar-41-2/xml/issue_meta.xml
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover missing
False
cover miss

# Notes for Checking and Updating the XML files ####

<!-- Caused by line breaks in source XML. Need to find *anything* after surname and string-name and remove accordingly. 
- all files need to be formatted and indented. 

Need to join and normalize to start:
F: \s+
R: ''

From there we need to fix formatting:
F: </surname> </
R: </surname></

F: </string-name> </
R: </string-name></

F: </given-names> </string-name>
R: </given-names></string-name>

F: </collab> </person-group>
R: </collab></person-group> -->

In [1]:
# Caused by line breaks in source XML. Need to find *anything* after surname and string-name and remove accordingly. 
# - all files need to be formatted and indented. 

# Need to join and normalize to start:
# F: \s+
# R: ''

# From there we need to fix formatting:
# F: </surname> </
# R: </surname></

# F: </string-name> </
# R: </string-name></

# F: </given-names> </string-name>
# R: </given-names></string-name>

# F: </collab> </person-group>
# R: </collab></person-group>

In [94]:
# import shutil
import glob, os
import logging


# logging.basicConfig(filename='log_file.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

def get_eps_files():
    # df_filter, unique_headings = get_doi_list('{}/{}'.format(input_path, issue_folder))

    delete_command = 'find \\output -name \'*.DS_Store*\' -delete'
    os.system(delete_command)
    for root, directory, files in os.walk(TEST_FOLDER):

        # Find the assets folder
        for d in directory:
            if d == 'assets':
                # Find the .eps.gz files
                assets_path = '{}/{}'.format(root, d)
                gz_files = get_gz_files(assets_path)
                print(gz_files)
#                 unzip_gz_files(gz_files)

def get_gz_files(assets_path):
    gz_files = [] # List to hold all the .eps.gz files
    for file in os.listdir(assets_path):
        if file.endswith(".eps.gz"):
            gz_files.append('{}/{}'.format(assets_path, file))
    return gz_files

def unzip_gz_files(gz_files):
    for file in gz_files:
        unzip_command = 'gzip -d {}'.format(file) # Will unzip and automatically delete .gz files
        os.system(unzip_command)
        print(file)




In [95]:
get_eps_files()

In [ ]:
# # import shutil
# import glob, os
# import logging


# # logging.basicConfig(filename='log_file.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)





# def run():
#     delete_command = 'find \\output -name \'*.DS_Store*\' -delete'
#     os.system(delete_command)
#     for root, directory, files in os.walk(output_path):

#         # Find the assets folder
#         for d in directory:
#             if d == 'assets':
#                 # Find the .eps.gz files
#                 assets_path = '{}/{}'.format(root, d)
#                 gz_files = get_gz_files(assets_path)
#                 unzip_gz_files(gz_files)


# def get_eps_files(assets_path):
#     gz_files = [] # List to hold all the .eps.gz files
#     for file in os.listdir(assets_path):
#         if file.endswith(".eps"):
#             gz_files.append('{}/{}'.format(assets_path, file))
#     return gz_files

# def delete_eps_files(gz_files):
#     for file in gz_files:
#         delete_command = 'rm {}'.format(file) # Will unzip and automatically delete .gz files
#         os.system(delete_command)
#         print(file)

# def zip_issues(folder_names):
#     for folder_name in folder_names:
#         zip_command = 'zip -r output/{}.zip output/{}'.format(folder_name, folder_name)
#         print(zip_command)
#         os.system(zip_command)

# run()
